In [1]:
import subprocess
from mlx_lm import load, generate

/Users/parikshith/projects/rag_pipeline/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
# prompt format
intstructions_string = f"""ShawGPT, functioning as a virtual data science \
consultant on YouTube, communicates in clear, accessible language, escalating \
to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, \
providing concise acknowledgments to brief expressions of gratitude or \
feedback, thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""

# define lambda function
prompt_builder = lambda comment: f'''<s>[INST] {intstructions_string} \n{comment} \n[/INST]\n'''

In [50]:
import os 
from dotenv import load_dotenv

# Load variables from the .env file
load_dotenv()

# Access them
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")


In [51]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)
index_name = "rag-pipeline-sources"

if index_name in pc.list_indexes().names():
    print("Using existing index")
else:
    print("Creating new index")
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
index = pc.Index(index_name)

Using existing index


In [14]:
import torch
from sentence_transformers import SentenceTransformer

device = "cuda" if torch.cuda.is_available() else "cpu"

embedding_model = SentenceTransformer(model_name_or_path="all-mpnet-base-v2",
                                      device=device)
def generate_embeddings(text_list):
  embeddings = embedding_model.encode(text_list, convert_to_tensor=True)
  return embeddings

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [15]:
def query_pinecone_db(query_text, k=5):
  query_embedding = generate_embeddings([query_text])[0].tolist()
  res = index.query(vector=query_embedding,top_k=k,include_values=True)
  res
  indices = [int(item["id"]) for item in res["matches"]]
  scores = [float(item["score"]) for item in res["matches"]]
  return indices,scores

In [ ]:
import json

def load_from_json(file_path: str) -> list[dict]:
    try:
        with open(file_path, 'r') as json_file:
            data = json.load(json_file)
        print(f"Data successfully loaded from {file_path}")
        return data
    except Exception as e:
        print(f"An error occurred while loading data: {e}")
        return []

In [23]:
source_file_path = "sources_and_chunks.json"
sources_and_chunks = load_from_json(source_file_path)
text_chunks = [item["sentence_chunk"] for item in sources_and_chunks]

Data successfully loaded from sources_and_chunks.json


In [24]:
query_text = "What are production activity management"
ids, scores = query_pinecone_db(query_text)
print(f"idexs: {ids}")
print(f"scores: {scores}")
for id in ids[:2]: # Printing top 2
  print("source: ", sources_and_chunks[id]["source"])
  print("source_file: ", sources_and_chunks[id]["source_file"])
  print("text: ", text_chunks[id])
  

idexs: [252, 70, 59, 413, 57]
scores: [0.599671, 0.569163203, 0.565872848, 0.561869442, 0.560454249]
source:  pdf
source_file:  /Users/parikshith/projects/rag_pipeline/rag_docs/pdf_files/Manufacturing Execution Systems Integration and Intelligence.pdf
text:  The next chapter presents just that. In helping with the exchange of information between the plant software systems, one standard that is being adapted by MES is open connectivity or OPC. OPC is a series of standards specifications. The first standard (originally called the OPC Specification and now called the Data Access Specification) resulted from the collaboration of a number of leading worldwide automation suppliers working in cooperation with Microsoft. Originally based on Microsoft's OLE COM (component object model) and DCOM (distributed component object model) technologies, the specification defined a standard set of objects, interfaces and methods for use in process control and manufacturing automation applications to faci

In [25]:
def get_context_items(query_text):
    ids, scores = query_pinecone_db(query_text)
    contexts = [text_chunks[id] for id in ids]
    return contexts

In [36]:
def generate_prompt(query: str) -> str:
    # Retrieve context items
    context_items = get_context_items(query_text=query)

    # Join context items into one dotted paragraph
    context = "- " + "\n- ".join(context_items)

    # Base prompt template
    base_prompt = f"""Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.
\nExample 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.
\nExample 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition, particularly the overconsumption of calories leading to obesity. Factors include a diet high in refined sugars and saturated fats, which can lead to insulin resistance, a condition where the body's cells do not respond effectively to insulin. Over time, the pancreas cannot produce enough insulin to manage blood sugar levels, resulting in type 2 diabetes. Additionally, excessive caloric intake without sufficient physical activity exacerbates the risk by promoting weight gain and fat accumulation, particularly around the abdomen, further contributing to insulin resistance.
\nExample 3:
Query: What is the importance of hydration for physical performance?
Answer: Hydration is crucial for physical performance because water plays key roles in maintaining blood volume, regulating body temperature, and ensuring the transport of nutrients and oxygen to cells. Adequate hydration is essential for optimal muscle function, endurance, and recovery. Dehydration can lead to decreased performance, fatigue, and increased risk of heat-related illnesses, such as heat stroke. Drinking sufficient water before, during, and after exercise helps ensure peak physical performance and recovery.
\nNow use the following context items to answer the user query:
{context}
\nRelevant passages: <extract relevant passages from the context here>
User query: {query}
Answer:"""
    
    return base_prompt

In [37]:
query_text = "What are production activity management"
final_prompt = generate_prompt(query=query_text)
print(final_prompt)


Based on the following context items, please answer the query.
Give yourself room to think by extracting relevant passages from the context before answering the query.
Don't return the thinking, only return the answer.
Make sure your answers are as explanatory as possible.
Use the following examples as reference for the ideal answer style.

Example 1:
Query: What are the fat-soluble vitamins?
Answer: The fat-soluble vitamins include Vitamin A, Vitamin D, Vitamin E, and Vitamin K. These vitamins are absorbed along with fats in the diet and can be stored in the body's fatty tissue and liver for later use. Vitamin A is important for vision, immune function, and skin health. Vitamin D plays a critical role in calcium absorption and bone health. Vitamin E acts as an antioxidant, protecting cells from damage. Vitamin K is essential for blood clotting and bone metabolism.

Example 2:
Query: What are the causes of type 2 diabetes?
Answer: Type 2 diabetes is often associated with overnutrition,

In [ ]:
model_path = "mlx-community/Llama-3.2-3B-Instruct-4bit"
prompt = prompt_builder("Great content, thank you!")
max_tokens = 140

# load model
model, tokenizer = load(model_path)

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

In [42]:
# generate response
def ask_rag_pipeline(user_query):
    final_prompt = generate_prompt(query=user_query) 
    response = generate(model, tokenizer, prompt=final_prompt, 
                                      max_tokens = max_tokens, 
                                      verbose=False)
    if isinstance(response, dict) and "response" in response:
        final_response = response["response"]
    else:
        # If `generate` returns raw text, ensure only the relevant part is extracted
        final_response = response.split("Answer:")[-1].strip()

# Return or use the final response
    return final_response

In [52]:
user_query = "What is supply chain?"
ask_rag_pipeline(user_query)

'Supply chain refers to the network of organizations, people, and activities involved in the production and delivery of a product or service. It encompasses the flow of goods, services, and information from raw materials to end customers. In the context of Manufacturing Execution Systems (MES), supply chain management involves the planning, coordination, and execution of activities related to the production and delivery of products, including inventory management, logistics, and transportation. MES solutions support supply chain management by providing real-time visibility into production processes, enabling data-driven decision-making, and automating tasks such as order management, inventory tracking, and material management. By optimizing supply chain operations, MES solutions can help manufacturers improve efficiency, reduce costs, and enhance'